In [10]:
%%HTML
<head>
    <title>Wordle Game</title>
    <style>
        body {
            text-align: center;
            font-family: Arial, sans-serif;
        }
        .position {
            font-weight: bold;
        }
        .message {
            margin-bottom: 10px;
        }

        {
            margin: 5px;
        }
    </style>
</head>


# Frontend HTML for Buttons and Input.
<body>
    <h1>Wordle Game</h1>
    <div id="menu">
        <h2>Welcome to the Wordle Game</h2>
        <p>Choose one of the following options:</p>
        <p>
            <button onclick="showRules()">Rules</button>
            <button onclick="playGame()">Play the Game</button>
            <button onclick="showLeaderboard()">View the Leaderboard</button>
            <button onclick="quitGame()">Quit</button>
        </p>
    </div>
    <div id="rules" style="display: none;">
        <h2>Rules</h2>
        <p>Your objective is to correctly guess a 5-letter word as quickly as possible given 5 attempts.</p>
        <p><button onclick="hideRules()">Back to menu</button></p>
    </div>
    <div id="game" style="display: none;">
        <h2>Play the Game</h2>
        <label for="nameInput">Enter your name:</label>
        <input type="text" id="nameInput">
        <br><br>
        <label for="difficultySelect">Select difficulty:</label>
        <select id="difficultySelect">
            <option value="easy">Easy</option>
            <option value="medium">Medium</option>
            <option value="hard">Hard</option>
        </select>
        <br><br>
        <button onclick="startGame()">Start</button>
        <p class="message" id="message"></p>
        <input type="text" id="guessInput" maxlength="5" size="5">
        <button onclick="checkGuess()">Check</button>
    </div>
    <div id="leaderboard" style="display: none;">
        <h2>Leaderboard</h2>
        <div id="scores"></div>
        <p><button onclick="hideLeaderboard()">Back to menu</button></p>
    </div>

# Javascript with a dictionary containing word list with differences in easy, medium, and hard.

# Variables for 

    <script>
        var wordList = {
            easy: ["apple", "baker", "candy", "drama", "eagle"],
            medium: ["juice", "image", "lemon", "melon", "music"],
            hard: ["blood", "shell", "heart", "swift", "unity"]
        };

        var gameStarted = false;
            # boolean variable: set to false and later updated if game is started.
        var word = "";
            # word (String): Stores the randomly selected word for the game that the player needs to guess.
        var correctPositions = [];
            #correctPositions (Array): Stores the indices of correct letters guessed by the player that are in the correct position within the word.
        var correctLetters = [];
            #correctLetters (Array): Stores the correct letters guessed by the player that are in the wrong position within the word.
        var guesses = 0;
            #guesses (Number): Keeps track of the number of attempts or guesses made by the player.
        var startTime = 0;
            #startTime (Number): Stores the timestamp when the game starts.

        function showRules() {
            document.getElementById("menu").style.display = "none";
            document.getElementById("rules").style.display = "block";
        }
# Function to show the rules
        function hideRules() {
            document.getElementById("rules").style.display = "none";
            document.getElementById("menu").style.display = "block";
        }
# Function to hide rules
        function playGame() {
            document.getElementById("menu").style.display = "none";
            document.getElementById("game").style.display = "block";
        }
# Function to play game
        function quitGame() {
            alert("Thanks for playing!");
        }

# Function to quit game
        function startGame() {
            var name = document.getElementById("nameInput").value;
            if (name.trim() === "") {
                alert("Please enter your name.");
                return;
            }
            gameStarted = true;
            document.getElementById("nameInput").disabled = true;
            document.getElementById("message").innerHTML = "Guess the 5-letter word:";
            document.getElementById("guessInput").focus();
            resetGame();
            startTime = Date.now();
        }
# Function to start the Game: 
# The startGame() function retrieves the player's name, checks if it's empty, and if not, it initializes the game by setting variables, disabling input fields, displaying instructions, resetting the game state, and recording the start time.


        function resetGame() {
            word = getWord();
            correctPositions = [];
            correctLetters = [];
            guesses = 0;
        }
# Resets the Game
        function getWord() {
            var difficultySelect = document.getElementById("difficultySelect");
            var difficulty = difficultySelect.value;
            var wordArray = wordList[difficulty];
            return wordArray[Math.floor(Math.random() * wordArray.length)];
        }
# Gets the word from the dictionary based on the difficulty level selected before. Randomly Chooses
       function checkGuess() {
    if (!gameStarted) {
        alert("Please start the game first.");
        return;
    }

    var guessInput = document.getElementById("guessInput");
    var guess = guessInput.value.toLowerCase();

    if (guess.length !== 5) {
        alert("Please enter a 5-letter word.");
        return;
    }

    guesses++;
# The checkGuess() function verifies if the game has started and if the player has entered a valid 5-letter word, displaying alerts if necessary, and increments the guess counter if the guess is valid.




    correctPositions = [];
    correctLetters = [];
# The correctPositions and correctLetters arrays are initialized.
    for (var i = 0; i < guess.length; i++) {
        if (guess[i] === word[i]) {
            correctPositions.push(i);
        } else if (word.indexOf(guess[i]) !== -1) {
            correctLetters.push(guess[i]);
        }
    }
# A loop iterates over each character in the player's guess to check for correct positions and correct letters.
    if (correctPositions.length === 5) {
        var endTime = Date.now();
        var totalTime = ((endTime - startTime) / 1000).toFixed(2);
        var message = "Congratulations, " + document.getElementById("nameInput").value + "! You guessed the word in " + totalTime + " seconds!";
        document.getElementById("message").innerHTML = message;
        gameStarted = false;
        document.getElementById("nameInput").disabled = false;
        document.getElementById("game").style.display = "none";
        document.getElementById("menu").style.display = "block";
        addScore(document.getElementById("nameInput").value, totalTime);
    } else {
        var result = "";
        for (var i = 0; i < guess.length; i++) {
            if (correctPositions.includes(i)) {
                result += "<span class='position'>" + guess[i] + "</span> (correct position) ";
            } else if (correctLetters.includes(guess[i])) {
                result += guess[i] + " (correct letter) ";
            } else {
                result += guess[i] + " ";
            }
        }
        guessInput.value = "";
        var message = "Incorrect guess. Attempts: " + guesses + "<br>" + result;
        document.getElementById("message").innerHTML = message;
    }
}
# If a character in the guess matches both in value and position with the secret word, its index is added to the correctPositions array.
# If a character in the guess matches a character in the word but not in the correct position, it is added to the correctLetters array.
# If all positions are correctly guessed (5 characters in total), the total time taken is calculated, a congratulatory message is displayed with the player's name and time, and the game state and UI are updated.
# If the guess is incorrect, a result string is constructed indicating correct positions and letters, the message is updated with the incorrect attempt count and result, and the guess input field is reset.
#
        function showLeaderboard() {
            document.getElementById("menu").style.display = "none";
            document.getElementById("leaderboard").style.display = "block";
            loadScores();
        }
# To Show Updated Leaderboad
        function hideLeaderboard() {
            document.getElementById("leaderboard").style.display = "none";
            document.getElementById("menu").style.display = "block";
        }
# To Hide Leaderboard

# The showLeaderboard() function displays the leaderboard section, hiding the "menu" element and showing the "leaderboard" element. It also updates the leaderboard by calling loadScores().


        function loadScores() {
            var scoresDiv = document.getElementById("scores");
            scoresDiv.innerHTML = "";
            var leaderboard = getLeaderboard();
            if (leaderboard.length === 0) {
                scoresDiv.innerHTML = "The leaderboard is empty.";
            } else {
                for (var i = 0; i < leaderboard.length; i++) {
                    var score = leaderboard[i];
                    var scoreItem = document.createElement("p");
                    scoreItem.innerText = (i + 1) + ". " + score.name + " - " + score.time + " seconds";
                    scoresDiv.appendChild(scoreItem);
                }
            }
        }
# The loadScores() function displays the leaderboard by clearing the current content, retrieving the leaderboard data, and dynamically creating HTML elements to display each score.
        function getLeaderboard() {
            var leaderboard = [];
            var leaderboardStr = localStorage.getItem("leaderboard");
            if (leaderboardStr) {
                leaderboard = JSON.parse(leaderboardStr);
            }
            return leaderboard;
        }
# The getLeaderboard() function retrieves the leaderboard data from local storage and returns it as an array.
        function addScore(name, time) {
            var leaderboard = getLeaderboard();
            leaderboard.push({ name: name, time: time });
            leaderboard.sort((a, b) => a.time - b.time);
            if (leaderboard.length > 10) {
                leaderboard.pop();
            }
            localStorage.setItem("leaderboard", JSON.stringify(leaderboard));
        }
# The addScore(name, time) function adds a new score to the leaderboard by retrieving the current leaderboard, adding the new score, sorting the leaderboard, and storing it back in local storage.
    </script>
</body>
</html>


Certainly! Let's dive into the technical parts of the code and explain each component:


1. JavaScript Variables: The code declares several variables at the beginning to store and manage game-related data. For example:
    wordList: An object containing words categorized by difficulty.
    gameStarted: A boolean flag to track whether the game has started.
    word: The selected word for the current game.
    correctPositions: An array to store the correct positions of the guessed letters.
    correctLetters: An array to store the correct letters that are in the word but not in the correct position.
    guesses: A counter to keep track of the number of guesses.
    startTime: A variable to store the start time of the game.

2. JavaScript Functions: The code defines various functions to handle different aspects of the game. Here are the functions and their responsibilities:
    showRules() and hideRules(): Control the visibility of the rules section.
    playGame(): Show the game section and hide the menu.
    quitGame(): Display an alert thanking the player for playing.
    startGame(): Initialize the game by retrieving the player's name, disabling the input field, setting up the game message, and resetting game variables.
    resetGame(): Reset the game variables and select a new word.
    getWord(): Retrieve a word based on the selected difficulty.
    checkGuess(): Validate and check the guessed word, update the game status, handle correct and incorrect guesses.
    showLeaderboard(): Display the leaderboard section and load the scores.
    hideLeaderboard(): Hide the leaderboard section and show the menu.
    loadScores(): Retrieve and display scores from local storage in the leaderboard section.
    getLeaderboard(): Retrieve the leaderboard scores from local storage.
    addScore(): Add a new score to the leaderboard, sort the leaderboard, and store it in local storage.

3. Event Handlers: The code uses event handlers like onclick to associate specific functions with button clicks. For example, when a button is clicked, the corresponding function is called to perform the associated action, such as showing rules, starting the game, or viewing the leaderboard.
   
4. Conditional Statements: Conditional statements such as if and else are used to check certain conditions and perform different actions based on the result. For example, the checkGuess() function checks if the game has started before processing the player's guess. If the game hasn't started, an alert is displayed.
   
5. Loops: The for loop is used in the checkGuess() function to iterate over the letters of the player's guess. It compares each letter with the correct word to determine correct positions and correct letters.
   
6. DOM Manipulation: DOM manipulation is used to access and modify HTML elements dynamically. The code uses functions like getElementById() to select elements by their IDs and then modifies their properties, such as changing the display style, updating innerHTML content, and disabling input fields. This allows the game to show or hide sections, display messages, and interact with user input.
   
7. Local Storage: The code utilizes the localStorage object to store and retrieve the leaderboard data. The getLeaderboard() function retrieves the leaderboard scores from local storage. The addScore() function adds a new score to the leaderboard, sorts it by time, and stores it in local storage. This allows the game to persist and display the leaderboard even after the page is refreshed or reopened.


The ++ operator is known as the increment operator in JavaScript. It is used to increment the value of a variable by 1.

```````
for (var i = 0; i < guess.length; i++) {
    if (guess[i] === word[i]) {
        correctPositions.push(i);
    } else if (word.indexOf(guess[i]) !== -1) {
        correctLetters.push(guess[i]);
    }
}
```````
This loop iterates over each character of the guess word entered by the player. It compares each character with the corresponding character at the same position in the word to check for correct positions. If a character in the guess matches the character in the word at the same position, the index i is pushed to the correctPositions array. If a character in the guess is not at the correct position but exists somewhere in the word, it is added to the correctLetters array.



````````
if (correctPositions.length === 5) {
    var endTime = Date.now();
    var totalTime = ((endTime - startTime) / 1000).toFixed(2);
    var message = "Congratulations, " + document.getElementById("nameInput").value + "! You guessed the word in " + totalTime + " seconds!";
    document.getElementById("message").innerHTML = message;
    gameStarted = false;
    document.getElementById("nameInput").disabled = false;
    document.getElementById("game").style.display = "none";
    document.getElementById("menu").style.display = "block";
    addScore(document.getElementById("nameInput").value, totalTime);
} else {
    var result = "";
    for (var i = 0; i < guess.length; i++) {
        if (correctPositions.includes(i)) {
            result += "<span class='position'>" + guess[i] + "</span> (correct position) ";
        } else if (correctLetters.includes(guess[i])) {
            result += guess[i] + " (correct letter) ";
        } else {
            result += guess[i] + " ";
        }
    }
    guessInput.value = "";
    var message = "Incorrect guess. Attempts: " + guesses + "<br>" + result;
    document.getElementById("message").innerHTML = message;
}
````````

If all characters in the guess are at the correct positions (correctPositions.length === 5), it means the player has guessed the word correctly. The code calculates the total time taken by subtracting the startTime from the current time and formatting it to two decimal places. It constructs a congratulatory message with the player's name and the total time, updates the message element in the HTML with this message, sets gameStarted to false, enables the name input field, hides the game div, displays the menu div, and calls the addScore() function to add the score to the leaderboard.

If the guess is incorrect, the code generates a result string that represents the result of the guess. It iterates over each character in the guess and checks if the character is in correctPositions or correctLetters. Based on the conditions, it constructs a string that highlights correct positions and correct letters. It clears the guess input field, constructs an incorrect guess message with the number of attempts (guesses) and the result string, updates the message element in the HTML with this message.

